In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
vocab = ''.join(sorted(list(set(text))))
vocab_size = len(vocab)
itos = list(vocab)
stoi = {itos[i] : i for i in range(vocab_size)}

n = int(0.9*len(text))
test_text = text[:n]
val_text = text[n:]
train_data = torch.tensor([stoi[c] for c in test_text])
val_data = torch.tensor([stoi[c] for c in val_text])


    

In [159]:
g = torch.manual_seed(1337)
block_size = 8
batch_size = 4
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,), generator=g)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [203]:
xb, yb = get_batch('train')
g = torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size: int) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B, T, C) = (4, 8, 65)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_tokens=100):
        for i in range(max_tokens):
            self(idx)

        
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
idx = torch.zeros((4,1), dtype=int)
for i in range(100):
    logits, _ = m(idx) #  (1, 1, 65)
    logits = logits[:, -1, :] # (1, 65)
    prob = F.softmax(logits, dim=-1) # (1, 65)
    next_word = torch.multinomial(prob, 1, generator=g)  # (1, 1)
    idx = torch.cat((idx, next_word), dim=1)

for batch in idx:
    print(''.join(itos[n] for n in batch))
# print(''.join(itos[n] for n in nums))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3

S
vviYDEsxyfqoioCp$NzbjuG&v&LWSbJ;hi?Lb$QY&ss;A'X3hiTOpTE;hJ?;phsZAUt$HZZ;ABjjup'Q&tRl,f.SPyhJFbZM?G

,OMkDCV&oj$-UybS
wfq-waeOvERmsEP f;H;-wvhs:CKSjHddZvEH-lH-Z?gFdfL& CWdzLERSGJkYKo-oimb?ERjulgC-m
pwJ

&Q:EJh$fRFuXuErIdbRmYhcmztgTEXWtlwa&Zy'P?a!PGLKIJoX:P N-w.oCKzPgOrUMkqUGAVOVE,'QKugXIfB!-gZ3cOdmnlIh
